In [1]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import wikipedia
from konlpy.tag import Kkma
from konlpy.utils import pprint
import matplotlib.pyplot as plt

In [2]:
wikipedia.set_lang('ko')
p = wikipedia.page(u'서울')

In [7]:
sentences = p.content.split('.')

In [9]:
kkma = Kkma()

In [113]:
vocas = {}
for s in sentences:
    for word,t in kkma.pos(s):
        if t == 'NNG':
            if not vocas.has_key(word):
                vocas[word] = len(vocas)
r_vocas = dict(zip(vocas.values(), vocas.keys()))

In [71]:
vocas_map = np.identity(len(vocas))
dataset = []
for s in sentences[:1]:
    nouns = []
    for word,t in kkma.pos(s):
        if t == 'NNG':
            nouns.append(word)
    for i in range(len(nouns)):
        if i-1 >= 0:
            j1 = vocas[nouns[i]]
            j2 = vocas[nouns[i-1]]
            dataset.append([ vocas_map[j1:j1+1][0], vocas_map[j2:j2+1]][0])
        if i+1 < len(nouns):
            j1 = vocas[nouns[i]]
            j2 = vocas[nouns[i+1]]
            dataset.append([ vocas_map[j1:j1+1][0], vocas_map[j2:j2+1]][0])

[ 0.  1.  0. ...,  0.  0.  0.]
[ 0.  0.  1. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]


In [67]:
np.random.shuffle(dataset)

In [87]:
train_x = []
train_y = []
for x,y in datset:
    train_x.append(x)
    train_y.append(y)
train_x = np.array(train_x).reshape(-1,len(vocas))
train_y = np.array(train_y).reshape(-1,len(vocas))

In [94]:
voca_size = len(vocas)
hidden_size = 2
x = tf.placeholder(dtype=tf.float32,shape=[None,voca_size])
y = tf.placeholder(dtype=tf.float32,shape=[None,voca_size])
w1 = tf.Variable(tf.random_normal(shape=[voca_size,hidden_size]))
b1 = tf.Variable(tf.zeros(shape=[1,hidden_size]))
w2 = tf.Variable(tf.random_normal(shape=[hidden_size,voca_size]))
b2 = tf.Variable(tf.zeros(shape=[1,voca_size]))
layer1 = tf.nn.relu(tf.matmul(x,w1) + b1)
layer2 = tf.nn.softmax(tf.matmul(layer1,w2) + b2)

cost = tf.reduce_mean(tf.square(-y*tf.log(layer2)))
grad = tf.train.AdamOptimizer(0.05).minimize(cost)
init = tf.global_variables_initializer()

In [129]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(300):
        c,_ = sess.run([cost,grad],{x:train_x,y:train_y})
        if i % 10 == 0:
            print c
    result = sess.run(layer2,{x:train_x[:10]})
    top_five = np.argsort(result)[::-1][:,:2]
    print top_five
    for i in range(len(train_x[:10])):
        print r_vocas[np.argmax(train_x[i])]
        for j in top_five[i]:
            print '  ', r_vocas[j]

0.0411391
0.0313566
0.0294096
0.0285125
0.0278883
0.0274003
0.0270229
0.0267335
0.0265117
0.0263486
0.0262173
0.026116
0.0260278
0.0259511
0.0258865
0.0258282
0.0257739
0.0257217
0.0256761
0.0256292
0.0255876
0.0255494
0.025509
nan
nan
nan
nan
nan
nan
nan


KeyboardInterrupt: 

In [126]:
np.argsort([3,4,1])[::-1]

array([1, 0, 2])